In [3]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,InputLayer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.optimizers import Adam
import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import pmdarima as pm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.var_model import VARResults

In [2]:
pip install plotly

DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jupyter-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 34.1 MB/s eta 0:00:000:00:01m eta 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
df= pd.read_csv('energy/bestFeatures.csv')
df

,datetime,hour,midnight_flag,part_of_day,humidity,feelslike,temp,sensor_temp,Consumption (kWh)
0,2023-11-02 19:00:00,19,False,2,97.650,5.30,8.50,20.913295,0.92
1,2023-11-02 19:30:00,19,False,2,96.495,5.10,8.45,20.971098,3.14
2,2023-11-02 20:00:00,20,False,2,95.340,4.90,8.40,21.028902,0.96
3,2023-11-02 20:30:00,20,False,2,94.745,4.85,8.35,21.052023,0.92
4,2023-11-02 21:00:00,21,False,6,94.150,4.80,8.30,21.075145,0.88
...,...,...,...,...,...,...,...,...,...
1300,2023-11-29 21:00:00,21,False,6,90.380,-4.60,-1.90,25.606936,0.88
1301,2023-11-29 21:30:00,21,False,6,89.740,-4.55,-1.85,25.583815,0.84
1302,2023-11-29 22:00:00,22,False,6,89.100,-4.50,-1.80,25.560694,0.05
1303,2023-11-29 22:30:00,22,False,6,90.105,-4.85,-1.65,25.606936,0.18


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1305 entries, 0 to 1304
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   datetime           1305 non-null   object 
 1   hour               1305 non-null   int64  
 2   midnight_flag      1305 non-null   bool   
 3   part_of_day        1305 non-null   int64  
 4   humidity           1305 non-null   float64
 5   feelslike          1305 non-null   float64
 6   temp               1305 non-null   float64
 7   sensor_temp        1305 non-null   float64
 8   Consumption (kWh)  1305 non-null   float64
dtypes: bool(1), float64(5), int64(2), object(1)
memory usage: 83.0+ KB


In [7]:
df.isnull().sum()

datetime             0
hour                 0
midnight_flag        0
part_of_day          0
humidity             0
feelslike            0
temp                 0
sensor_temp          0
Consumption (kWh)    0
dtype: int64

In [8]:

df['datetime'] = pd.to_datetime(df['datetime'])

# Split 'datetime' into date, month, year, and time columns
df['date'] = df['datetime'].dt.date
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year
df['time'] = df['datetime'].dt.time

# Display the updated DataFrame
print(df[['date', 'month', 'year', 'time']])


            date  month  year      time
0     2023-11-02     11  2023  19:00:00
1     2023-11-02     11  2023  19:30:00
2     2023-11-02     11  2023  20:00:00
3     2023-11-02     11  2023  20:30:00
4     2023-11-02     11  2023  21:00:00
...          ...    ...   ...       ...
1300  2023-11-29     11  2023  21:00:00
1301  2023-11-29     11  2023  21:30:00
1302  2023-11-29     11  2023  22:00:00
1303  2023-11-29     11  2023  22:30:00
1304  2023-11-29     11  2023  23:00:00

[1305 rows x 4 columns]


In [9]:
df

,datetime,hour,midnight_flag,part_of_day,humidity,feelslike,temp,sensor_temp,Consumption (kWh),date,month,year,time
0,2023-11-02 19:00:00,19,False,2,97.650,5.30,8.50,20.913295,0.92,2023-11-02,11,2023,19:00:00
1,2023-11-02 19:30:00,19,False,2,96.495,5.10,8.45,20.971098,3.14,2023-11-02,11,2023,19:30:00
2,2023-11-02 20:00:00,20,False,2,95.340,4.90,8.40,21.028902,0.96,2023-11-02,11,2023,20:00:00
3,2023-11-02 20:30:00,20,False,2,94.745,4.85,8.35,21.052023,0.92,2023-11-02,11,2023,20:30:00
4,2023-11-02 21:00:00,21,False,6,94.150,4.80,8.30,21.075145,0.88,2023-11-02,11,2023,21:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,2023-11-29 21:00:00,21,False,6,90.380,-4.60,-1.90,25.606936,0.88,2023-11-29,11,2023,21:00:00
1301,2023-11-29 21:30:00,21,False,6,89.740,-4.55,-1.85,25.583815,0.84,2023-11-29,11,2023,21:30:00
1302,2023-11-29 22:00:00,22,False,6,89.100,-4.50,-1.80,25.560694,0.05,2023-11-29,11,2023,22:00:00
1303,2023-11-29 22:30:00,22,False,6,90.105,-4.85,-1.65,25.606936,0.18,2023-11-29,11,2023,22:30:00


In [11]:

# Assuming the data is stored in a DataFrame named 'df'
# Convert 'datetime' column to pandas datetime format
df['datetime'] = pd.to_datetime(df['datetime'])

# Filter data for daytime (6:00 to 20:00) and nighttime (20:00 to 6:00)
daytime_data = df[(df['datetime'].dt.hour >= 6) & (df['datetime'].dt.hour < 20)]
nighttime_data = df[(df['datetime'].dt.hour >= 20) | (df['datetime'].dt.hour < 6)]

# Calculate average energy consumption for daytime and nighttime
avg_daytime_consumption = daytime_data['Consumption (kWh)'].mean()
avg_nighttime_consumption = nighttime_data['Consumption (kWh)'].mean()

# Create a DataFrame for plotting
data = {'Time of Day': ['Daytime', 'Nighttime'],
        'Average Energy Consumption (kWh)': [avg_daytime_consumption, avg_nighttime_consumption]}

# Create a bar plot using Plotly
fig = px.bar(data, x='Time of Day', y='Average Energy Consumption (kWh)',
             title='Average Energy Consumption During Daytime and Nighttime',
             color='Time of Day', color_discrete_map={'Daytime': 'lightgreen', 'Nighttime': 'skyblue'})

# Show the plot
fig.show()


In [12]:

# Assuming your DataFrame is named 'df'

# Convert 'datetime' column to datetime format
df['datetime'] = pd.to_datetime(df['datetime'])

# Group by date and calculate the mean consumption for each day
daily_consumption = df.groupby('date')['Consumption (kWh)'].mean().reset_index()

# Plotting energy consumption with day using Plotly
fig = px.line(daily_consumption, x='date', y='Consumption (kWh)', title='Average Daily Energy Consumption')
fig.update_xaxes(title='Date', tickangle=45)
fig.update_yaxes(title='Average Consumption (kWh)')
fig.show()


In [13]:
import plotly.express as px
import pandas as pd

# Assuming the dataset is loaded into a DataFrame named 'data'

# Filter the data for daytime records (6 AM to 6 PM) and nighttime records (6 PM to 6 AM)
daytime_data = df[(df['hour'] >= 6) & (df['hour'] <= 18)]
nighttime_data = df[((df['hour'] >= 18) & (df['hour'] <= 23)) | ((df['hour'] >= 0) & (df['hour'] <= 6))]

# Group by date and sum the daytime and nighttime energy consumption separately
daytime_consumption = daytime_data.groupby('date')['Consumption (kWh)'].sum().reset_index()
nighttime_consumption = nighttime_data.groupby('date')['Consumption (kWh)'].sum().reset_index()

# Plotting both daytime and nighttime energy consumption using Plotly line chart
fig = px.line()

# Add daytime energy consumption line
fig.add_scatter(x=daytime_consumption['date'], y=daytime_consumption['Consumption (kWh)'],
                mode='lines', name='Daytime', line=dict(color='orange'))

# Add nighttime energy consumption line
fig.add_scatter(x=nighttime_consumption['date'], y=nighttime_consumption['Consumption (kWh)'],
                mode='lines', name='Nighttime', line=dict(color='blue'))

# Update chart labels and title
fig.update_layout(title='Daytime vs Nighttime Energy Consumption',
                  xaxis=dict(title='Date'),
                  yaxis=dict(title='Energy Consumption (kWh)'),
                  xaxis_tickangle=-45)

fig.show()


In [14]:
import plotly.express as px
import pandas as pd
# Assuming 'df' is your DataFrame with the relevant columns
numeric_columns = df.select_dtypes(include='number')

# Calculate correlation matrix
correlation_matrix = numeric_columns.corr()

# Display the correlation matrix


# Assuming 'correlation_matrix' is the calculated correlation matrix
# If not, calculate it using your data

# Sample correlation matrix (replace this with your actual correlation matrix)
# correlation_matrix = your_data_frame.corr()

# Create a heatmap using Plotly
fig = px.imshow(correlation_matrix,
                labels=dict(color="Correlation"),
                x=correlation_matrix.columns,
                y=correlation_matrix.columns,
                color_continuous_scale='Viridis')

# Customize the layout
fig.update_layout(title="Correlation Heatmap",
                  width=800,
                  height=600)

fig.show()
